In [ ]:
from google.colab import drive

# Monta o Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from google.colab import auth
auth.authenticate_user()

In [3]:
import subprocess
import re

# ---------- CONFIGURAÇÃO ----------
BUCKET = "brats-final-training-818"
base_dir = f"gs://{BUCKET}/nnUNet_raw_database/Task001_MyBraTS"

imagesTr = f"{base_dir}/imagesTr"
labelsTr = f"{base_dir}/labelsTr"

# Criar pastas (gsutil aceita mkdir a partir de 2023)
subprocess.run(["gsutil", "mkdir", "-p", imagesTr])
subprocess.run(["gsutil", "mkdir", "-p", labelsTr])

# ---------- LISTAR TODOS OS ARQUIVOS ----------
def gcs_ls(path):
    result = subprocess.run(
        ["gsutil", "ls", path],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    return [x for x in result.stdout.strip().split("\n") if x]

files = gcs_ls(f"{base_dir}/*.nii.gz")

print(f"Encontrados {len(files)} arquivos no bucket.")

# ---------- SEPARAÇÃO ----------
for src in files:
    fname = src.split("/")[-1]

    if "seg" in fname.lower():
        # É label
        dst = f"{labelsTr}/{fname}"
        print(f"Label:  {fname}")
        subprocess.run(["gsutil", "-q", "mv", src, dst])

    else:
        # É imagem
        dst = f"{imagesTr}/{fname}"
        print(f"Imagem: {fname}")
        subprocess.run(["gsutil", "-q", "mv", src, dst])

print("\n✅ Separação concluída! Agora o bucket tem imagesTr/ e labelsTr/")


Encontrados 4003 arquivos no bucket.
Label:  BraTS-GLI-00005-100-seg.nii.gz
Imagem: BraTS-GLI-00005-100_0000.nii.gz
Imagem: BraTS-GLI-00005-100_0001.nii.gz
Imagem: BraTS-GLI-00005-100_0002.nii.gz
Imagem: BraTS-GLI-00005-100_0003.nii.gz
Label:  BraTS-GLI-00006-100-seg.nii.gz
Imagem: BraTS-GLI-00006-100_0000.nii.gz
Imagem: BraTS-GLI-00006-100_0001.nii.gz
Imagem: BraTS-GLI-00006-100_0002.nii.gz
Imagem: BraTS-GLI-00006-100_0003.nii.gz
Label:  BraTS-GLI-00008-100-seg.nii.gz
Imagem: BraTS-GLI-00008-100_0000.nii.gz
Imagem: BraTS-GLI-00008-100_0001.nii.gz
Imagem: BraTS-GLI-00008-100_0002.nii.gz
Imagem: BraTS-GLI-00008-100_0003.nii.gz
Label:  BraTS-GLI-00009-100-seg.nii.gz
Imagem: BraTS-GLI-00009-100_0000.nii.gz
Imagem: BraTS-GLI-00009-100_0001.nii.gz
Imagem: BraTS-GLI-00009-100_0002.nii.gz
Imagem: BraTS-GLI-00009-100_0003.nii.gz
Label:  BraTS-GLI-00020-100-seg.nii.gz
Imagem: BraTS-GLI-00020-100_0000.nii.gz
Imagem: BraTS-GLI-00020-100_0001.nii.gz
Imagem: BraTS-GLI-00020-100_0002.nii.gz
Imagem: 

KeyboardInterrupt: 

In [3]:
import subprocess
import random
import re

# ---------- CONFIGURAÇÃO ----------
BUCKET = "brats-final-training-818"
base_dir = f"gs://{BUCKET}/Task001_MyBraTS"

imagesTr = f"{base_dir}/imagesTr"
labelsTr = f"{base_dir}/labelsTr"
imagesTs = f"{base_dir}/imagesTs"
labelsTs = f"{base_dir}/labelsTs"

# Pastas são criadas automaticamente no primeiro upload/move
subprocess.run(["gsutil", "mkdir", "-p", imagesTs])
subprocess.run(["gsutil", "mkdir", "-p", labelsTs])

# ---------- FUNÇÃO: LISTAR ----------
def gcs_ls(path):
    result = subprocess.run(
        ["gsutil", "ls", path],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    return [x for x in result.stdout.strip().split("\n") if x]

# ---------- LISTA ARQUIVOS ----------
files = gcs_ls(f"{imagesTr}/*.nii.gz")

# Extrai IDs do paciente:
# Ex: "BraTS-GLI-02837_0000.nii.gz" → "BraTS-GLI-02837"
patients = sorted(set(re.sub(r"_[0-9]{4}$", "", f.split("/")[-1][:-7]) for f in files))

print("➡️ Total de pacientes encontrados:", len(patients))

# ---------- GERAR SPLIT ----------
random.seed(42)
test_ratio = 0.1
num_test = int(len(patients) * test_ratio)
test_patients = random.sample(patients, num_test)

print(f"🧠 Pacientes selecionados p/ teste ({num_test}):")
print(test_patients)

# ---------- MOVER ARQUIVOS ----------
for pid in test_patients:
    print(f"\n📦 Movendo paciente {pid}...")

    # mover modalidades
    modality_files = gcs_ls(f"{imagesTr}/{pid}*.nii.gz")
    for src in modality_files:
        dst = f"{imagesTs}/{src.split('/')[-1]}"
        subprocess.run(["gsutil", "mv", src, dst])
        print("  → imagem:", src.split("/")[-1])

    # mover labels
    label_files = gcs_ls(f"{labelsTr}/{pid}*.nii.gz")
    for src in label_files:
        dst = f"{labelsTs}/{src.split('/')[-1]}"
        subprocess.run(["gsutil", "mv", src, dst])
        print("  → label:", src.split("/")[-1])

print("\n✅ Split de treino e teste criado com sucesso no bucket!")

➡️ Total de pacientes encontrados: 818
🧠 Pacientes selecionados p/ teste (81):
['BraTS-GLI-02867-100', 'BraTS-GLI-02149-100', 'BraTS-GLI-00512-100', 'BraTS-GLI-02989-100', 'BraTS-GLI-02369-100', 'BraTS-GLI-02331-100', 'BraTS-GLI-02297-100', 'BraTS-GLI-02186-101', 'BraTS-GLI-02983-100', 'BraTS-GLI-02137-100', 'BraTS-GLI-02905-100', 'BraTS-GLI-02988-100', 'BraTS-GLI-02758-100', 'BraTS-GLI-02120-100', 'BraTS-GLI-02809-100', 'BraTS-GLI-02592-100', 'BraTS-GLI-00547-100', 'BraTS-GLI-00533-100', 'BraTS-GLI-02127-100', 'BraTS-GLI-02287-100', 'BraTS-GLI-02313-100', 'BraTS-GLI-02710-100', 'BraTS-GLI-02826-100', 'BraTS-GLI-00518-100', 'BraTS-GLI-02775-100', 'BraTS-GLI-02259-100', 'BraTS-GLI-02954-100', 'BraTS-GLI-02878-100', 'BraTS-GLI-02934-100', 'BraTS-GLI-02587-100', 'BraTS-GLI-02289-100', 'BraTS-GLI-02630-100', 'BraTS-GLI-02808-100', 'BraTS-GLI-02374-100', 'BraTS-GLI-00033-100', 'BraTS-GLI-03013-100', 'BraTS-GLI-02211-100', 'BraTS-GLI-02930-100', 'BraTS-GLI-02489-100', 'BraTS-GLI-02207-100', 

In [3]:
!gsutil -m mv gs://brats-final-training-818/nnUNet_raw_database/Task001_MyBraTS/** gs://brats-final-training-818/nnUNet_raw_database/Dataset001_MyBraTS/

A saída de streaming foi truncada nas últimas 5000 linhas.
Removing gs://brats-final-training-818/Task001_MyBraTS/imagesTr/BraTS-GLI-02579-100_0000.nii.gz...
Copying gs://brats-final-training-818/Task001_MyBraTS/imagesTr/BraTS-GLI-02580-100_0002.nii.gz [Content-Type=application/octet-stream]...
Removing gs://brats-final-training-818/Task001_MyBraTS/imagesTr/BraTS-GLI-02579-100_0003.nii.gz...
Copying gs://brats-final-training-818/Task001_MyBraTS/imagesTr/BraTS-GLI-02580-100_0003.nii.gz [Content-Type=application/octet-stream]...
Removing gs://brats-final-training-818/Task001_MyBraTS/imagesTr/BraTS-GLI-02580-100_0000.nii.gz...
Removing gs://brats-final-training-818/Task001_MyBraTS/imagesTr/BraTS-GLI-02580-100_0002.nii.gz...
Removing gs://brats-final-training-818/Task001_MyBraTS/imagesTr/BraTS-GLI-02580-100_0001.nii.gz...
Copying gs://brats-final-training-818/Task001_MyBraTS/imagesTr/BraTS-GLI-02581-101_0000.nii.gz [Content-Type=application/octet-stream]...
Copying gs://brats-final-trainin

In [2]:
import shutil
from pathlib import Path
import random
import re

# Caminhos principais
base_dir = Path("/content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_50/nnUNet_raw_data_base/nnUNet_raw_data/Task001_MyBraTS")
imagesTr = base_dir / "imagesTr"
labelsTr = base_dir / "labelsTr"
imagesTs = base_dir / "imagesTs"
labelsTs = base_dir / "labelsTs"

# Garante que as pastas de teste existam
imagesTs.mkdir(exist_ok=True)
labelsTs.mkdir(exist_ok=True)

# Define uma seed para reprodução
random.seed(42)

# Extrai prefixos únicos (IDs de pacientes)
patients = sorted(set(f.name.rsplit('-', 1)[0] for f in imagesTr.glob("*.nii.gz")))

# Define proporção de teste
test_ratio = 0.1
num_test = int(len(patients) * test_ratio)
test_patients = random.sample(patients, num_test)

print(f"➡️ Total de pacientes: {len(patients)}")
print(f"🧠 Pacientes para teste ({num_test}): {test_patients}")

# Move os arquivos
for pid in test_patients:
    # move todas as modalidades
    for img_file in imagesTr.glob(f"{pid}*.nii.gz"):
        shutil.move(str(img_file), str(imagesTs / img_file.name))
    # move label correspondente
    for label_file in labelsTr.glob(f"{pid}*.nii.gz"):
      shutil.move(str(label_file), str(labelsTs / label_file.name))

print("✅ Split de treino e teste criado com sucesso!")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_50/nnUNet_raw_data_base/nnUNet_raw_data/Task001_MyBraTS/imagesTs'